# Style Instruction Evaluation Framework

This notebook evaluates how well derived style instructions enable style replication, compared to alternative approaches.

## Methodology

For each gold standard text:
1. **Neutralize**: Rewrite text in bland journalistic prose, preserving rhetorical moves and argumentative structure while removing distinctive stylistic choices
2. **Reconstruct**: Generate text using 4 different methods (M stochastic runs each):
   - Generic baseline
   - Few-shot learning
   - Author name prompting
   - Derived style instructions
3. **Judge (Blind Comparative)**: Judge ranks all 4 reconstructions from 1-4 based on similarity to original
   - **Blind evaluation**: Judge sees only anonymous labels (Text A, B, C, D) - no method names
   - **Ranking**: 1 = most similar, 2 = second, 3 = third, 4 = least similar
   - **Order randomized**: Position of methods varies across samples to eliminate bias
4. **Aggregate**: Analyze rankings to determine which method best captures style

## Key Features

- **Crash resilient**: All LLM responses saved to SQLite immediately
- **Resume support**: Can restart after failures, skips completed work
- **Blind evaluation**: Eliminates judge bias by hiding method names
- **Comparative ranking**: More informative than binary comparisons

## Updated Approach (StyleNeutralization)

Unlike pure content extraction (StyleFlattening), this version uses **StyleNeutralization** which:
- Preserves complete rhetorical structure (concessions, qualifications, emphasis patterns)
- Maintains logical connectors and argumentative moves
- Removes only distinctive stylistic execution (word choices, sentence rhythm, figurative language)
- Produces full-length neutral rewrites (~80-100% of original length)

This gives reconstruction methods the rhetorical scaffolding while leaving room for stylistic variation.

### Install Libraries and Check

In [ ]:
!pip install -r requirements.txt

In [ ]:
try:
    import litellm
    print('Providers\n=========')
    print('* ' + '\n* '.join(litellm.LITELLM_CHAT_PROVIDERS))
    litellm.drop_params = True
except ImportError as e:
    print(f"✗ Cannot import litellm: {e}")

## Setup and Configuration

In [ ]:
import os
import json
from pathlib import Path
import pandas as pd
from datetime import datetime

### Initialize Base Objects
The base objects part of the current project library (`belletrist`) are initialized. They are:
* `LLM`: the LLM object.
* `LLMConfig`: the configuration of the LLM object, such as what model to use.
* `PromptMaker`: generates prompts from templates and variables
* `DataSampler`: retrieves and samples text at a source directory

These will implement text transformations by LLMs part of the evaluation process. They build on the third-party LLMs, which we furthermore split into LLMs for text reconstruction and text judging, the key parameters for which are set below.

In [ ]:
#model_reconstruction_string = 'together_ai/Qwen/Qwen3-235B-A22B-Instruct-2507-tput'
#model_reconstruction_api_key_env_var = 'TOGETHER_AI_API_KEY'
#model_reconstruction_string = 'anthropic/claude-sonnet-4-5-20250929'
#model_reconstruction_api_key_env_var = 'ANTHROPIC_API_KEY'
#model_reconstruction_string = 'openai/gpt-5.1-2025-11-13'
#model_reconstruction_api_key_env_var = 'OPENAI_API_KEY'
#model_reconstruction_string = 'together_ai/moonshotai/Kimi-K2-Instruct'
#model_reconstruction_api_key_env_var = 'TOGETHER_AI_API_KEY'
model_reconstruction_string = 'mistral/mistral-large-2512'
model_reconstruction_api_key_env_var = 'MISTRAL_API_KEY'
model_judge_string = 'anthropic/claude-sonnet-4-5-20250929'
model_judge_api_key_env_var = 'ANTHROPIC_API_KEY'
#model_judge_string = 'mistral/mistral-large-2512'
#model_judge_api_key_env_var = 'MISTRAL_API_KEY'

In [ ]:
from belletrist import PromptMaker, DataSampler, StyleEvaluationStore, SegmentStore

# ============================================================================
# CONFIGURATION - Modify these parameters before running
# ============================================================================

# Data paths
DATA_PATH = Path(os.getcwd()) / "data" / "russell"
SEGMENT_DB_PATH = Path(os.getcwd()) / "segments_mistral.db"
EVALUATION_DB_PATH = Path(os.getcwd()) / "style_eval_statistical_s_mistral_r_mistral_j_anthropic.db"

# Methods for this experiment (must be exactly 4)
METHODS = ['generic', 'fewshot', 'author', 'agent_statistical']

# ============================================================================

# Validate configuration
if not DATA_PATH.exists():
    raise FileNotFoundError(
        f"Data directory not found: {DATA_PATH}\n"
        f"Please ensure the data directory exists."
    )

if not SEGMENT_DB_PATH.exists():
    raise FileNotFoundError(
        f"Segment database not found: {SEGMENT_DB_PATH}\n"
        f"The 'agent_fewshot' method requires a segment catalog.\n"
        f"Please run 'python runs/style_retrieval.py' first to build the catalog."
    )

# Initialize components
prompt_maker = PromptMaker()

sampler = DataSampler(data_path=DATA_PATH.resolve())

store = StyleEvaluationStore(
    EVALUATION_DB_PATH,
    methods=METHODS
)

print(f"✓ Data path: {DATA_PATH}")
print(f"✓ Segment database: {SEGMENT_DB_PATH}")
print(f"✓ Evaluation database: {EVALUATION_DB_PATH}")
print(f"✓ Configured methods: {METHODS}")

In [ ]:
store.reset('reconstructions_and_judgments')

In [ ]:
from belletrist import LLM, LLMConfig

reconstruction_llm = LLM(LLMConfig(
    model=model_reconstruction_string,
    api_key=os.environ.get(model_reconstruction_api_key_env_var)
))
judge_llm = LLM(LLMConfig(
    model=model_judge_string,
    api_key=os.environ.get(model_judge_api_key_env_var)
))

### Sample Test Data and Few-Shot Data
The reconstruction method tests build on gold standard texts. The test also includes few-shot prompting with the gold standard texts. In order to not skew the tests, no few-shot examples can overlap with the test texts.

In [ ]:
n_sample = 10
m_paragraphs_per_sample = 3
n_few_shot_sample = 3

In [ ]:
#test_texts = []
#for _ in range(n_sample):
#    test_texts.append(sampler.sample_segment(p_length=m_paragraphs_per_sample))

In [10]:
quality_texts_deterministic = [
    sampler.get_paragraph_chunk(file_index=0, paragraph_range=slice(9, 9+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=0, paragraph_range=slice(29, 29+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=0, paragraph_range=slice(131, 131+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=1, paragraph_range=slice(13, 13+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=1, paragraph_range=slice(39, 39+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=1, paragraph_range=slice(192, 192+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=2, paragraph_range=slice(20, 20+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=2, paragraph_range=slice(43, 43+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=2, paragraph_range=slice(146, 146+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=3, paragraph_range=slice(7, 7+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=3, paragraph_range=slice(73, 73+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=3, paragraph_range=slice(202, 202+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=3, paragraph_range=slice(285, 285+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=4, paragraph_range=slice(4, 4+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=4, paragraph_range=slice(67, 67+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=4, paragraph_range=slice(124, 124+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=5, paragraph_range=slice(6, 6+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=5, paragraph_range=slice(119, 119+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=5, paragraph_range=slice(301, 301+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=6, paragraph_range=slice(23, 23+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=6, paragraph_range=slice(75, 75+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=6, paragraph_range=slice(152, 152+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=6, paragraph_range=slice(198, 198+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=6, paragraph_range=slice(271, 271+m_paragraphs_per_sample)),
]
reindex = [0, 5, 10, 15, 20, 1, 6, 11, 16, 21, 2, 7, 12, 17, 22, 3, 8, 13, 18, 23, 4, 9, 14, 19]
test_texts = [
    quality_texts_deterministic[i] for i in reindex[:n_sample]
]

In [11]:
from random import choice
few_shot_texts = []
while len(few_shot_texts) < n_few_shot_sample:
    #p = sampler.sample_segment(p_length=m_paragraphs_per_sample)
    p = choice(quality_texts_deterministic)

    # Check if p overlaps with any test text
    # Two segments overlap if they're from the same file AND their paragraph ranges overlap
    # Ranges [a, b) and [c, d) overlap if: a < d AND c < b
    has_overlap = any(
        p.file_index == test_seg.file_index and
        p.paragraph_start < test_seg.paragraph_end and
        test_seg.paragraph_start < p.paragraph_end
        for test_seg in test_texts
    )

    if not has_overlap:
        few_shot_texts.append(p)

## Create the Test Transformation Objects
The combination of prompt and LLM leads to the following operators in the test chain:
* **Style Neutralizer**, which given a text rewrites it in bland journalistic prose while preserving rhetorical structure and argumentative moves.
* **Reconstructor, LLM House Style**, which given neutralized content expands it into a complete text with the "house style" of the LLM employed for the reconstruction.
* **Reconstructor, Few Shot**, which given neutralized content expands it into a complete text with a few text excerpts on unrelated topics as style guide.
* **Reconstructor, LLM Author Model**, which given neutralized content expands it into a complete text with the named author's style as the LLM conceives it without any other guidance.
* **Reconstructor, Style Instruction**, which given neutralized content expands it into a complete text following the detailed style instruction, as derived from previous analysis.

In [12]:
# Import from style-retrieval (no models/ subdirectory)
from belletrist.style_evaluation_models import (
    MethodMapping,
    StyleJudgmentComparative
)
from belletrist.prompts import (
    StyleNeutralizationConfig,  # Changed from StyleFlatteningConfig
    StyleReconstructionGenericConfig,
    StyleReconstructionFewShotConfig,
    StyleReconstructionAuthorConfig,
    StyleJudgeComparativeConfig
)

# Configuration
n_runs = 2
n_judge_runs = 1
AUTHOR_NAME = "Bertrand Russell"

# Reconstructor configs (agent_fewshot handled separately)
RECONSTRUCTORS_CFGS = {
    'generic': StyleReconstructionGenericConfig,
    'fewshot': StyleReconstructionFewShotConfig,
    'author': StyleReconstructionAuthorConfig,
    'agent_statistical': None  # Special case: uses agent_rewriter.agent_rewrite_statistical
}

# Reconstructor kwargs
RECONSTRUCTORS_KWARGS = {
    'generic': {},
    'fewshot': {'few_shot_examples': [seg.text for seg in few_shot_texts]},
    'author': {'author_name': AUTHOR_NAME},
    'agent_statistical': {}  # Handled separately
}

print(f"✓ Configuration: {n_runs} runs per sample, {len(METHODS)} reconstruction methods")
print(f"✓ Methods: {', '.join(METHODS)}")

✓ Configuration: 2 runs per sample, 4 reconstruction methods
✓ Methods: generic, fewshot, author, agent_statistical


## Evaluation Pipeline

### Step 1: Content Neutralization

Rewrite each test sample in neutral journalistic prose, preserving rhetorical structure and argumentative moves while removing distinctive stylistic choices:

In [13]:
# Step 1: Save samples and neutralize content (preserving rhetorical structure)
print("=== Step 1: Neutralizing Style (Preserving Argument Structure) ===\n")

for k_text, test_sample in enumerate(test_texts):
    sample_id = f"sample_{k_text:03d}"
    
    # Skip if already saved
    if store.get_sample(sample_id):
        print(f"✓ {sample_id} already neutralized (skipping)")
        continue
    
    print(f"Neutralizing {sample_id}...", end=" ")
    
    # Neutralize content (rewrite in bland journalistic prose)
    neutralize_prompt = prompt_maker.render(
        StyleNeutralizationConfig(text=test_sample.text)
    )
    neutralized = reconstruction_llm.complete(neutralize_prompt)
    
    # Save to store with provenance
    source_info = f"File {test_sample.file_index}, para {test_sample.paragraph_start}-{test_sample.paragraph_end}"
    store.save_sample(
        sample_id=sample_id,
        original_text=test_sample.text,
        flattened_content=neutralized.content,  # Still using 'flattened_content' column for compatibility
        flattening_model=neutralized.model,
        source_info=source_info
    )
    
    print(f"✓ ({len(neutralized.content)} chars)")

print(f"\n✓ All samples neutralized and saved to store")

=== Step 1: Neutralizing Style (Preserving Argument Structure) ===

✓ sample_000 already neutralized (skipping)
Neutralizing sample_001... ✓ (3005 chars)
Neutralizing sample_002... ✓ (2032 chars)
Neutralizing sample_003... ✓ (1672 chars)
Neutralizing sample_004... ✓ (2693 chars)
Neutralizing sample_005... ✓ (4103 chars)
Neutralizing sample_006... ✓ (4136 chars)
Neutralizing sample_007... ✓ (3196 chars)
Neutralizing sample_008... ✓ (2284 chars)
Neutralizing sample_009... ✓ (3410 chars)

✓ All samples neutralized and saved to store


In [28]:
print('====ORIGINAL===')
print(store.get_sample('sample_004')['original_text'])
print('\n\n====NEUTRALIZED (BLAND REWRITE)====')
print(store.get_sample('sample_004')['flattened_content'])

====ORIGINAL===
In the preceding chapter we saw that there are two sorts of knowledge:
knowledge of things, and knowledge of truths. In this chapter we shall
be concerned exclusively with knowledge of things, of which in turn we
shall have to distinguish two kinds. Knowledge of things, when it is
of the kind we call knowledge by _acquaintance_, is essentially simpler
than any knowledge of truths, and logically independent of knowledge
of truths, though it would be rash to assume that human beings ever,
in fact, have acquaintance with things without at the same time knowing
some truth about them. Knowledge of things by _description_, on the
contrary, always involves, as we shall find in the course of the present
chapter, some knowledge of truths as its source and ground. But first of
all we must make clear what we mean by 'acquaintance' and what we mean
by 'description'.

We shall say that we have _acquaintance_ with anything of which we are
directly aware, without the intermediary of a

### Step 2: Reconstruction

Generate reconstructions using all 4 methods, with M stochastic runs each:

In [15]:
# Step 2: Generate reconstructions (with crash resume and agent_fewshot support)
print("=== Step 2: Generating Reconstructions ===\n")

for sample_id in store.list_samples():
    sample = store.get_sample(sample_id)
    print(f"\n{sample_id}:")
    
    for run in range(n_runs):
        print(f"  Run {run}:")
        
        # Check which methods need reconstruction
        for method in METHODS:
            if store.has_reconstruction(sample_id, run, method):
                print(f"    ✓ {method:15s} (already done)")
                continue
            
            # Special handling for agent_statistical
            if method == 'agent_statistical':
                from belletrist.agent_rewriter import agent_rewrite_statistical
                
                # Initialize LLM for rewriting (no planning phase!)
                rewriting_llm = LLM(LLMConfig(
                    model=model_reconstruction_string,
                    api_key=os.environ.get(model_reconstruction_api_key_env_var),
                    temperature=0.7,  # Creative rewriting
                    max_tokens=8192  # Full rewrites need generous token budget
                ))
                
                with SegmentStore(SEGMENT_DB_PATH) as segment_store:
                    reconstructed_text = agent_rewrite_statistical(
                        flattened_content=sample['flattened_content'],
                        segment_store=segment_store,
                        rewriting_llm=rewriting_llm,
                        prompt_maker=prompt_maker,
                        num_examples=10  # Statistical selection
                    )
                
                # Save reconstruction
                store.save_reconstruction(
                    sample_id=sample_id,
                    run=run,
                    method=method,
                    reconstructed_text=reconstructed_text,
                    model=model_reconstruction_string
                )
                print(f"    ✓ {method:15s} ({len(reconstructed_text)} chars)")
            else:
                # Standard reconstruction
                config = RECONSTRUCTORS_CFGS[method](
                    content_summary=sample['flattened_content'],
                    **RECONSTRUCTORS_KWARGS[method]
                )
                prompt = prompt_maker.render(config)
                response = reconstruction_llm.complete(prompt)
                
                # Save immediately (crash resilient!)
                store.save_reconstruction(
                    sample_id=sample_id,
                    run=run,
                    method=method,
                    reconstructed_text=response.content,
                    model=response.model
                )
                print(f"    ✓ {method:15s} ({len(response.content)} chars)")

stats = store.get_stats()
print(f"\n✓ Generated {stats['n_reconstructions']} total reconstructions")
print(f"✓ Configured methods: {stats['configured_methods']}")

=== Step 2: Generating Reconstructions ===


sample_000:
  Run 0:
    ✓ generic         (3750 chars)
    ✓ fewshot         (4010 chars)
    ✓ author          (5489 chars)
You are given a text written in neutral, straightforward journalistic prose. The text preserves all argumentative structure, rhetorical moves, and content, but uses bland, unmarked language without distinctive stylistic choices.

Your task is to rewrite this text with the writing style demonstrated and explained in the examples below.

**CRITICAL REQUIREMENTS:**
- Match the STYLE of the examples (rhythm, vocabulary, tone, sentence structure), not their formatting choices or topics.
- Preserve all argumentative structure and rhetorical moves from the neutral text.
- Transform the neutral prose by applying the stylistic techniques shown in the examples.
- If the examples are plain prose with no titles/headers/formatting, write plain prose.
- Do NOT add meta-commentary, preambles like "Here is the passage...", or postscr

    ✓ agent_statistical (3361 chars)
  Run 1:
    ✓ generic         (3650 chars)
    ✓ fewshot         (3257 chars)
    ✓ author          (6027 chars)
You are given a text written in neutral, straightforward journalistic prose. The text preserves all argumentative structure, rhetorical moves, and content, but uses bland, unmarked language without distinctive stylistic choices.

Your task is to rewrite this text with the writing style demonstrated and explained in the examples below.

**CRITICAL REQUIREMENTS:**
- Match the STYLE of the examples (rhythm, vocabulary, tone, sentence structure), not their formatting choices or topics.
- Preserve all argumentative structure and rhetorical moves from the neutral text.
- Transform the neutral prose by applying the stylistic techniques shown in the examples.
- If the examples are plain prose with no titles/headers/formatting, write plain prose.
- Do NOT add meta-commentary, preambles like "Here is the passage...", or postscripts explaining your

    ✓ agent_statistical (4032 chars)

sample_001:
  Run 0:
    ✓ generic         (3928 chars)
    ✓ fewshot         (4063 chars)
    ✓ author          (4000 chars)
You are given a text written in neutral, straightforward journalistic prose. The text preserves all argumentative structure, rhetorical moves, and content, but uses bland, unmarked language without distinctive stylistic choices.

Your task is to rewrite this text with the writing style demonstrated and explained in the examples below.

**CRITICAL REQUIREMENTS:**
- Match the STYLE of the examples (rhythm, vocabulary, tone, sentence structure), not their formatting choices or topics.
- Preserve all argumentative structure and rhetorical moves from the neutral text.
- Transform the neutral prose by applying the stylistic techniques shown in the examples.
- If the examples are plain prose with no titles/headers/formatting, write plain prose.
- Do NOT add meta-commentary, preambles like "Here is the passage...", or postscripts ex

    ✓ agent_statistical (4093 chars)
  Run 1:
    ✓ generic         (4372 chars)
    ✓ fewshot         (3642 chars)
    ✓ author          (5061 chars)
You are given a text written in neutral, straightforward journalistic prose. The text preserves all argumentative structure, rhetorical moves, and content, but uses bland, unmarked language without distinctive stylistic choices.

Your task is to rewrite this text with the writing style demonstrated and explained in the examples below.

**CRITICAL REQUIREMENTS:**
- Match the STYLE of the examples (rhythm, vocabulary, tone, sentence structure), not their formatting choices or topics.
- Preserve all argumentative structure and rhetorical moves from the neutral text.
- Transform the neutral prose by applying the stylistic techniques shown in the examples.
- If the examples are plain prose with no titles/headers/formatting, write plain prose.
- Do NOT add meta-commentary, preambles like "Here is the passage...", or postscripts explaining your

    ✓ agent_statistical (4256 chars)

sample_002:
  Run 0:
    ✓ generic         (3104 chars)
    ✓ fewshot         (2052 chars)
    ✓ author          (3614 chars)
You are given a text written in neutral, straightforward journalistic prose. The text preserves all argumentative structure, rhetorical moves, and content, but uses bland, unmarked language without distinctive stylistic choices.

Your task is to rewrite this text with the writing style demonstrated and explained in the examples below.

**CRITICAL REQUIREMENTS:**
- Match the STYLE of the examples (rhythm, vocabulary, tone, sentence structure), not their formatting choices or topics.
- Preserve all argumentative structure and rhetorical moves from the neutral text.
- Transform the neutral prose by applying the stylistic techniques shown in the examples.
- If the examples are plain prose with no titles/headers/formatting, write plain prose.
- Do NOT add meta-commentary, preambles like "Here is the passage...", or postscripts ex

...

    ✓ agent_statistical (3614 chars)

sample_008:
  Run 0:
    ✓ generic         (3069 chars)
    ✓ fewshot         (2654 chars)
    ✓ author          (3083 chars)
You are given a text written in neutral, straightforward journalistic prose. The text preserves all argumentative structure, rhetorical moves, and content, but uses bland, unmarked language without distinctive stylistic choices.

Your task is to rewrite this text with the writing style demonstrated and explained in the examples below.

**CRITICAL REQUIREMENTS:**
- Match the STYLE of the examples (rhythm, vocabulary, tone, sentence structure), not their formatting choices or topics.
- Preserve all argumentative structure and rhetorical moves from the neutral text.
- Transform the neutral prose by applying the stylistic techniques shown in the examples.
- If the examples are plain prose with no titles/headers/formatting, write plain prose.
- Do NOT add meta-commentary, preambles like "Here is the passage...", or postscripts ex

    ✓ agent_statistical (2510 chars)
  Run 1:
    ✓ generic         (2461 chars)
    ✓ fewshot         (2443 chars)
    ✓ author          (3446 chars)
You are given a text written in neutral, straightforward journalistic prose. The text preserves all argumentative structure, rhetorical moves, and content, but uses bland, unmarked language without distinctive stylistic choices.

Your task is to rewrite this text with the writing style demonstrated and explained in the examples below.

**CRITICAL REQUIREMENTS:**
- Match the STYLE of the examples (rhythm, vocabulary, tone, sentence structure), not their formatting choices or topics.
- Preserve all argumentative structure and rhetorical moves from the neutral text.
- Transform the neutral prose by applying the stylistic techniques shown in the examples.
- If the examples are plain prose with no titles/headers/formatting, write plain prose.
- Do NOT add meta-commentary, preambles like "Here is the passage...", or postscripts explaining your

    ✓ agent_statistical (2369 chars)

sample_009:
  Run 0:
    ✓ generic         (3879 chars)
    ✓ fewshot         (3924 chars)
    ✓ author          (4647 chars)
You are given a text written in neutral, straightforward journalistic prose. The text preserves all argumentative structure, rhetorical moves, and content, but uses bland, unmarked language without distinctive stylistic choices.

Your task is to rewrite this text with the writing style demonstrated and explained in the examples below.

**CRITICAL REQUIREMENTS:**
- Match the STYLE of the examples (rhythm, vocabulary, tone, sentence structure), not their formatting choices or topics.
- Preserve all argumentative structure and rhetorical moves from the neutral text.
- Transform the neutral prose by applying the stylistic techniques shown in the examples.
- If the examples are plain prose with no titles/headers/formatting, write plain prose.
- Do NOT add meta-commentary, preambles like "Here is the passage...", or postscripts ex

    ✓ agent_statistical (3475 chars)
  Run 1:
    ✓ generic         (4762 chars)
    ✓ fewshot         (4155 chars)
    ✓ author          (4766 chars)
You are given a text written in neutral, straightforward journalistic prose. The text preserves all argumentative structure, rhetorical moves, and content, but uses bland, unmarked language without distinctive stylistic choices.

Your task is to rewrite this text with the writing style demonstrated and explained in the examples below.

**CRITICAL REQUIREMENTS:**
- Match the STYLE of the examples (rhythm, vocabulary, tone, sentence structure), not their formatting choices or topics.
- Preserve all argumentative structure and rhetorical moves from the neutral text.
- Transform the neutral prose by applying the stylistic techniques shown in the examples.
- If the examples are plain prose with no titles/headers/formatting, write plain prose.
- Do NOT add meta-commentary, preambles like "Here is the passage...", or postscripts explaining your

    ✓ agent_statistical (3224 chars)

✓ Generated 80 total reconstructions
✓ Configured methods: ['generic', 'fewshot', 'author', 'agent_statistical']


In [16]:
config = RECONSTRUCTORS_CFGS['fewshot'](
                content_summary=sample['flattened_content'],
                **RECONSTRUCTORS_KWARGS['fewshot']
            )
prompt = prompt_maker.render(config)
print(prompt)

You are given a text written in neutral, straightforward journalistic prose. The text preserves all argumentative structure, rhetorical moves, and content, but uses bland, unmarked language without distinctive stylistic choices.

Your task is to rewrite this text with the writing style demonstrated in the examples below.

**CRITICAL REQUIREMENTS:**
- Match the STYLE of the examples (rhythm, vocabulary, tone, sentence structure), not their formatting choices or topics.
- Preserve all argumentative structure and rhetorical moves from the neutral text.
- Transform the neutral prose by applying the stylistic techniques shown in the examples.
- If the examples are plain prose with no titles/headers/formatting, write plain prose.
- Do NOT add meta-commentary, preambles like "Here is the passage...", or postscripts explaining your work.
- Simply write naturally in this style.

**EXAMPLE TEXTS (demonstrating the target style):**

---

The mystic insight begins with the sense of a mystery unvei

In [29]:
reconstructions = store.get_reconstructions('sample_004', 0)
for reconstructor in reconstructions.keys():
    print(f"{reconstructor.upper()}\n===================")
    print(f"\n{reconstructions.get(reconstructor)}\n\n")

AGENT_STATISTICAL

The distinction drawn in the preceding chapter—between knowledge of things and knowledge of truths—has left us with a task before us: to examine the former, which, though seemingly more straightforward, unfolds into two forms of its own. The first, *acquaintance*, stands as the more primitive, the more immediate, and, in its essence, the more certain. It does not, in logic, depend upon the second, yet to suppose that one might ever possess acquaintance with a thing without also holding some truth about it would be a folly. The second, *knowledge by description*, is a more elaborate affair, resting always upon a foundation of truths, as we shall see in due course. But before we proceed further, we must first clarify what we mean when we speak of *acquaintance* and *description*—terms that, though familiar in their common use, here demand a precision they do not always receive.

Acquaintance, then, is direct awareness—unmediated, uninferred, unadorned. When one looks u

### Step 3: Judging

Compare each reconstruction against the original using the judge LLM:

In [18]:
# Import from style-retrieval (no models/ subdirectory)
from belletrist.style_evaluation_models import StyleJudgmentComparative
from belletrist.prompts import StyleJudgeComparativeConfig

# Step 3: Comparative blind judging (with crash resume and judge consistency testing)
print("=== Step 3: Comparative Blind Judging ===\n")

for sample_id in store.list_samples():
    sample = store.get_sample(sample_id)
    print(f"\n{sample_id}:")
    
    for reconstruction_run in range(n_runs):
        print(f"  Reconstruction run {reconstruction_run}:")
        
        # Get all 4 reconstructions ONCE
        reconstructions = store.get_reconstructions(sample_id, reconstruction_run)
        if len(reconstructions) != 4:
            print(f"    ✗ Missing reconstructions (found {len(reconstructions)}/4)")
            continue
        
        # Create mapping ONCE per reconstruction_run (deterministic seed for reproducibility)
        mapping = store.create_random_mapping(seed=hash(f"{sample_id}_{reconstruction_run}"))
        
        # Build prompt ONCE per reconstruction_run (SAME prompt for all judge runs)
        judge_config = StyleJudgeComparativeConfig(
            original_text=sample['original_text'],
            reconstruction_text_a=reconstructions[mapping.text_a],
            reconstruction_text_b=reconstructions[mapping.text_b],
            reconstruction_text_c=reconstructions[mapping.text_c],
            reconstruction_text_d=reconstructions[mapping.text_d]
        )
        judge_prompt = prompt_maker.render(judge_config)
        
        # Judge the SAME reconstructions with the SAME prompt multiple times
        for judge_run in range(n_judge_runs):
            if store.has_judgment(sample_id, reconstruction_run, judge_run):
                print(f"    Judge run {judge_run}: ✓ Already judged (skipping)")
                continue
            
            print(f"    Judge run {judge_run}: Judging...", end=" ")
            
            # Get structured JSON judgment with schema enforcement
            try:
                response = judge_llm.complete_with_schema(judge_prompt, StyleJudgmentComparative)
                judgment = response.content  # Already validated Pydantic instance
                
                # Save judgment with both reconstruction_run and judge_run
                store.save_judgment(
                    sample_id=sample_id,
                    reconstruction_run=reconstruction_run,
                    judgment=judgment,
                    mapping=mapping,
                    judge_model=response.model,
                    judge_run=judge_run
                )
                print(f"✓ (confidence: {judgment.confidence})")
                
            except Exception as e:
                print(f"✗ Error: {e}")

stats = store.get_stats()
print(f"\n✓ Completed {stats['n_judgments']} judgments")

=== Step 3: Comparative Blind Judging ===


sample_000:
  Reconstruction run 0:
    Judge run 0: Judging... ✓ (confidence: high)
  Reconstruction run 1:
    Judge run 0: Judging... ✓ (confidence: high)

sample_001:
  Reconstruction run 0:
    Judge run 0: Judging... ✓ (confidence: high)
  Reconstruction run 1:
    Judge run 0: Judging... ✓ (confidence: high)

sample_002:
  Reconstruction run 0:
    Judge run 0: Judging... ✓ (confidence: high)
  Reconstruction run 1:
    Judge run 0: Judging... ✓ (confidence: high)

sample_003:
  Reconstruction run 0:
    Judge run 0: Judging... ✓ (confidence: high)
  Reconstruction run 1:
    Judge run 0: Judging... ✓ (confidence: high)

sample_004:
  Reconstruction run 0:
    Judge run 0: Judging... ✓ (confidence: high)
  Reconstruction run 1:
    Judge run 0: Judging... ✓ (confidence: high)

sample_005:
  Reconstruction run 0:
    Judge run 0: Judging... ✓ (confidence: high)
  Reconstruction run 1:
    Judge run 0: Judging... ✓ (confidence: high)

sa

## Results Analysis

In [19]:
# Export results to DataFrame
print("=== Exporting Results ===\n")

# Export from store (resolves anonymous rankings to methods)
df = store.to_dataframe()

print(f"Total judgments: {len(df)}")
print(f"Samples: {df['sample_id'].nunique()}")
print(f"Reconstruction runs per sample: {df.groupby('sample_id')['reconstruction_run'].nunique().mean():.1f}")
print(f"Judge runs per reconstruction: {df.groupby(['sample_id', 'reconstruction_run'])['judge_run'].nunique().mean():.1f}")

# Show first few rows (with dynamic method columns)
print(f"\n=== Sample Results ===\n")
display_cols = ['sample_id', 'reconstruction_run', 'judge_run'] + [f'ranking_{m}' for m in METHODS] + ['confidence']
print(df[display_cols].head(25))

# Export to CSV
output_file = f"style_eval_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
df.to_csv(output_file, index=False)
print(f"\n✓ Results saved to {output_file}")

=== Exporting Results ===

Total judgments: 20
Samples: 10
Reconstruction runs per sample: 2.0
Judge runs per reconstruction: 1.0

=== Sample Results ===

     sample_id  reconstruction_run  judge_run  ranking_generic  \
0   sample_000                   0          0                4   
1   sample_000                   1          0                3   
2   sample_001                   0          0                4   
3   sample_001                   1          0                4   
4   sample_002                   0          0                4   
5   sample_002                   1          0                4   
6   sample_003                   0          0                4   
7   sample_003                   1          0                4   
8   sample_004                   0          0                4   
9   sample_004                   1          0                4   
10  sample_005                   0          0                4   
11  sample_005                   1          0        

In [20]:
# Analyze judge consistency across multiple judgments of same reconstructions
print("=== Judge Consistency Analysis ===\n")

if 'judge_run' in df.columns and df['judge_run'].nunique() > 1:
    # For each (sample_id, reconstruction_run), check ranking variance across judge_runs
    consistency_results = []
    
    for (sample_id, recon_run), group in df.groupby(['sample_id', 'reconstruction_run']):
        if len(group) > 1:  # Only if multiple judge runs exist
            for method in ['generic', 'fewshot', 'author', 'instructions']:
                col = f'ranking_{method}'
                ranks = group[col].values
                variance = ranks.std()
                mean_rank = ranks.mean()
                
                consistency_results.append({
                    'sample_id': sample_id,
                    'reconstruction_run': recon_run,
                    'method': method,
                    'mean_rank': mean_rank,
                    'std_dev': variance,
                    'min_rank': ranks.min(),
                    'max_rank': ranks.max(),
                    'rank_range': ranks.max() - ranks.min(),
                    'n_judgments': len(ranks)
                })
    
    if consistency_results:
        consistency_df = pd.DataFrame(consistency_results)
        
        print("Method-level consistency (across all samples):")
        method_consistency = consistency_df.groupby('method').agg({
            'std_dev': 'mean',
            'rank_range': 'mean'
        }).round(2)
        method_consistency.columns = ['Avg Std Dev', 'Avg Rank Range']
        print(method_consistency)
        print("\n(Lower values = more consistent)")
        
        print("\n\nMost inconsistent cases (judge disagreed most):")
        inconsistent = consistency_df.nlargest(10, 'rank_range')[
            ['sample_id', 'reconstruction_run', 'method', 'min_rank', 'max_rank', 'rank_range']
        ]
        print(inconsistent.to_string(index=False))
        
        print("\n\nMost consistent cases (judge agreed most):")
        consistent = consistency_df.nsmallest(10, 'std_dev')[
            ['sample_id', 'reconstruction_run', 'method', 'mean_rank', 'std_dev']
        ]
        print(consistent.to_string(index=False))
        
else:
    print("⚠ Only one judge_run per reconstruction. Set n_judge_runs > 1 to test consistency.")

=== Judge Consistency Analysis ===

⚠ Only one judge_run per reconstruction. Set n_judge_runs > 1 to test consistency.


In [ ]:
print(df.loc[0,'reasoning'])

In [24]:
# Analyze ranking distributions (dynamic for configured methods)
print("=== Ranking Distribution by Method ===\n")

for method in METHODS:
    col = f'ranking_{method}'
    print(f"\n{method.upper()}:")
    ranking_counts = df[col].value_counts().sort_index()
    for rank in [1, 2, 3, 4]:
        count = ranking_counts.get(rank, 0)
        pct = (count / len(df) * 100) if len(df) > 0 else 0
        print(f"  Rank {rank}: {count:3d} ({pct:5.1f}%)")

print("\n=== Confidence Distribution ===\n")
print(df['confidence'].value_counts())

=== Ranking Distribution by Method ===


GENERIC:
  Rank 1:   0 (  0.0%)
  Rank 2:   0 (  0.0%)
  Rank 3:   2 ( 10.0%)
  Rank 4:  18 ( 90.0%)

FEWSHOT:
  Rank 1:   1 (  5.0%)
  Rank 2:   7 ( 35.0%)
  Rank 3:  11 ( 55.0%)
  Rank 4:   1 (  5.0%)

AUTHOR:
  Rank 1:   3 ( 15.0%)
  Rank 2:  11 ( 55.0%)
  Rank 3:   5 ( 25.0%)
  Rank 4:   1 (  5.0%)

AGENT_STATISTICAL:
  Rank 1:  16 ( 80.0%)
  Rank 2:   2 ( 10.0%)
  Rank 3:   2 ( 10.0%)
  Rank 4:   0 (  0.0%)

=== Confidence Distribution ===

confidence
high    20
Name: count, dtype: int64


In [ ]:
# Calculate method performance metrics (dynamic for configured methods)
print("=== Method Performance Metrics ===\n")

# Calculate mean ranking for each method (lower is better: 1 = best, 4 = worst)
mean_rankings = {}
for method in METHODS:
    col = f'ranking_{method}'
    mean_rankings[method] = df[col].mean()

# Sort by mean ranking (best first)
sorted_methods = sorted(mean_rankings.items(), key=lambda x: x[1])

print("Average Ranking (lower is better):")
for i, (method, mean_rank) in enumerate(sorted_methods, 1):
    # Count how often this method ranked 1st
    first_place = (df[f'ranking_{method}'] == 1).sum()
    first_place_pct = (first_place / len(df) * 100) if len(df) > 0 else 0
    
    print(f"{i}. {method:15s}: {mean_rank:.2f} (1st place: {first_place}/{len(df)} = {first_place_pct:.1f}%)")

# Win rate (percentage of times ranked 1st or 2nd)
print("\nTop-2 Rate (ranked 1st or 2nd):")
for method in METHODS:
    col = f'ranking_{method}'
    top2 = ((df[col] == 1) | (df[col] == 2)).sum()
    top2_pct = (top2 / len(df) * 100) if len(df) > 0 else 0
    print(f"  {method:15s}: {top2}/{len(df)} = {top2_pct:.1f}%")

## Detailed Inspection

Examine individual samples, reconstructions, and judge reasoning:

In [25]:
# View judge reasoning for a specific sample/run
INSPECT_SAMPLE = 'sample_001'
INSPECT_RUN = 0
INSPECT_JUDGE_RUN = 0

# Query judgment directly from database
judgment = store.conn.execute("""
    SELECT * FROM comparative_judgments
    WHERE sample_id=? AND reconstruction_run=? AND judge_run=?
""", (INSPECT_SAMPLE, INSPECT_RUN, INSPECT_JUDGE_RUN)).fetchone()

if judgment:
    print(f"=== JUDGE REASONING: {INSPECT_SAMPLE}, Reconstruction Run {INSPECT_RUN}, Judge Run {INSPECT_JUDGE_RUN} ===\n")
    
    # Build mapping to show which label = which method
    label_to_method = {
        'A': judgment['method_text_a'],
        'B': judgment['method_text_b'],
        'C': judgment['method_text_c'],
        'D': judgment['method_text_d']
    }
    
    label_to_rank = {
        'A': judgment['ranking_text_a'],
        'B': judgment['ranking_text_b'],
        'C': judgment['ranking_text_c'],
        'D': judgment['ranking_text_d']
    }
    
    print("ANONYMOUS LABELS → METHODS:")
    for label in ['A', 'B', 'C', 'D']:
        method = label_to_method[label]
        rank = label_to_rank[label]
        print(f"  Text {label} = {method:25s} → Rank {rank}")
    
    print(f"\nConfidence: {judgment['confidence']}")
    print(f"Judge Model: {judgment['judge_model']}")
    
    # Show reasoning
    print(f"\n{'REASONING':-^80}")
    print(judgment['reasoning'])
else:
    print(f"No judgment found for {INSPECT_SAMPLE}, run {INSPECT_RUN}, judge run {INSPECT_JUDGE_RUN}")

=== JUDGE REASONING: sample_001, Reconstruction Run 0, Judge Run 0 ===

ANONYMOUS LABELS → METHODS:
  Text A = author                    → Rank 2
  Text B = generic                   → Rank 4
  Text C = fewshot                   → Rank 3
  Text D = agent_statistical         → Rank 1

Confidence: high
Judge Model: claude-sonnet-4-5-20250929

-----------------------------------REASONING------------------------------------
Let me analyze each text's stylistic similarity to Russell's original voice.

**Original Russell's distinctive features:**
- Measured, clear prose with elegant phrasing but not overwrought
- Dry wit and understated humor (the courtiers anecdote, the comment about monarchs)
- Patient explanatory style that guides the reader through complex ideas
- Balance between conversational accessibility and intellectual sophistication
- Smooth transitions between historical anecdote and philosophical exposition
- Natural rhythm that doesn't call attention to itself
- Subtle irony wi

In [22]:
# Compare rankings across all runs for a specific sample
INSPECT_SAMPLE = 'sample_000'

# Query all judgments for this sample from database
judgments = store.conn.execute("""
    SELECT * FROM comparative_judgments
    WHERE sample_id=?
    ORDER BY reconstruction_run, judge_run
""", (INSPECT_SAMPLE,)).fetchall()

if judgments:
    print(f"=== RANKING CONSISTENCY FOR {INSPECT_SAMPLE} ===\n")
    print(f"Total judgments: {len(judgments)}\n")
    
    # Show rankings by run
    print(f"{'Run':<6} {'Judge':<6} ", end='')
    for method in METHODS:
        print(f"{method[:12]:<14}", end='')
    print()
    print("-" * 70)
    
    # Collect rankings for mean calculation
    rankings_by_method = {method: [] for method in METHODS}
    
    for j in judgments:
        # Build method->rank mapping for this judgment
        method_ranks = {}
        for label in ['a', 'b', 'c', 'd']:
            method = j[f'method_text_{label}']
            rank = j[f'ranking_text_{label}']
            method_ranks[method] = rank
        
        # Print row
        print(f"{j['reconstruction_run']:<6} {j['judge_run']:<6} ", end='')
        for method in METHODS:
            rank = method_ranks[method]
            rankings_by_method[method].append(rank)
            print(f"{rank:<14}", end='')
        print()
    
    # Show mean rankings
    print("\n\nMean rankings across all runs:")
    import statistics
    for method in METHODS:
        ranks = rankings_by_method[method]
        mean_rank = statistics.mean(ranks)
        std_rank = statistics.stdev(ranks) if len(ranks) > 1 else 0.0
        print(f"  {method:25s}: {mean_rank:.2f} ± {std_rank:.2f}")
else:
    print(f"No judgments found for {INSPECT_SAMPLE}")

=== RANKING CONSISTENCY FOR sample_000 ===

Total judgments: 2

Run    Judge  generic       fewshot       author        agent_statis  
----------------------------------------------------------------------
0      0      4             2             1             3             
1      0      3             4             2             1             


Mean rankings across all runs:
  generic                  : 3.50 ± 0.71
  fewshot                  : 3.00 ± 1.41
  author                   : 1.50 ± 0.71
  agent_statistical        : 2.00 ± 1.41


In [26]:
# View all reconstructions for a specific sample and run
INSPECT_SAMPLE = 'sample_001'
INSPECT_RUN = 0

print(f"=== RECONSTRUCTIONS FOR {INSPECT_SAMPLE}, RUN {INSPECT_RUN} ===\n")

sample = store.get_sample(INSPECT_SAMPLE)
reconstructions = store.get_reconstructions(INSPECT_SAMPLE, INSPECT_RUN)

print(f"{'ORIGINAL':-^80}")
print(sample['original_text'])
print("\n\n")

for method in METHODS:
    print(f"{method.upper():-^80}")
    print(reconstructions[method])
    print(f"\n({len(reconstructions[method])} chars)\n")

=== RECONSTRUCTIONS FOR sample_001, RUN 0 ===

------------------------------------ORIGINAL------------------------------------
The infinitesimal played formerly a great part in mathematics. It was
introduced by the Greeks, who regarded a circle as differing
infinitesimally from a polygon with a very large number of very small
equal sides. It gradually grew in importance, until, when Leibniz
invented the Infinitesimal Calculus, it seemed to become the
fundamental notion of all higher mathematics. Carlyle tells, in his
_Frederick the Great_, how Leibniz used to discourse to Queen Sophia
Charlotte of Prussia concerning the infinitely little, and how she
would reply that on that subject she needed no instruction--the
behaviour of courtiers had made her thoroughly familiar with it. But
philosophers and mathematicians--who for the most part had less
acquaintance with courts--continued to discuss this topic, though
without making any advance. The Calculus required continuity, and
continuity 

In [ ]:
# View the reconstruction prompt for a specific method
INSPECT_SAMPLE = 'sample_000'
INSPECT_METHOD = 'fewshot'  # Change to 'generic', 'fewshot', 'author' (agent_statistical uses statistical workflow)

sample = store.get_sample(INSPECT_SAMPLE)

print(f"=== RECONSTRUCTION PROMPT: {INSPECT_METHOD.upper()} ===\n")

if INSPECT_METHOD == 'agent_statistical':
    print("⚠ The 'agent_statistical' method uses statistical few-shot selection,")
    print("  not a single prompt. See belletrist/agent_rewriter.py for details.")
elif INSPECT_METHOD in RECONSTRUCTORS_CFGS:
    config = RECONSTRUCTORS_CFGS[INSPECT_METHOD](
        content_summary=sample['flattened_content'],
        **RECONSTRUCTORS_KWARGS[INSPECT_METHOD]
    )
    prompt = prompt_maker.render(config)
    print(prompt)
else:
    print(f"Unknown method: {INSPECT_METHOD}")

## Next Steps

TODO: Add analysis cells for:
- Statistical significance testing (e.g., Friedman test for rankings)
- Visualization of results (bar charts, violin plots)
- Pairwise method comparisons
- Effect size calculations

The detailed inspection cells above allow you to:
- View judge reasoning with method labels revealed
- Check ranking consistency across multiple runs
- Compare all reconstructions side-by-side
- Examine the prompts used for each method